In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow import keras

import numpy as np
from keras import layers
import tensorflow_addons as tfa
import random
import matplotlib.pyplot as plt
AUTOTUNE = tf.data.AUTOTUNE
import cv2

In [ ]:
root_dir = "tfrecords_position"
batch_size = 128


def parse_tfrecord_fn(example):
    feature_description = {
      "image": tf.io.FixedLenFeature([], tf.string),
      "path": tf.io.FixedLenFeature([], tf.string),
      "category_id": tf.io.FixedLenFeature([], tf.int64),        
    }
    example = tf.io.parse_single_example(example, feature_description)    
    example["image"] = tf.io.decode_png(example["image"], channels=3)       
    return example


# cria arquivos de treinamento
def prepare_sample_train(features):
    rand = tf.random.uniform([5])
    noise = tf.random.normal(shape=(224,224,3), mean=0.0, stddev=8, dtype=tf.float32)    
    image = tf.cast(features["image"], dtype=tf.float32)
    image = tf.image.resize(image, size=(224, 224))   
    
    #format(x, 'b').zfill(n)
    augment_op = format(np.random.randint(0, 31), 'b').zfill(5)
    
    # BRIGHTNESS
    if augment_op[0] == '1':
        aux_image = tf.image.random_brightness(image, 0.5)    
        image = ((1-rand[1])*image + rand[1]*aux_image)    
 
    # HUE
    if augment_op[1] == '1':
        aux_image = tf.image.random_hue(image, 0.08)
        image = ((1-rand[2])*image + rand[2]*aux_image)
    
    # CONTRAST
    if augment_op[2] == '1':
        aux_image = tf.clip_by_value(tf.image.random_contrast(image, 0.8, 1.5), 0, 255)
        image = ((1-rand[3])*image + rand[3]*aux_image)

    # CROP
    if augment_op[3] == '1':
        image = tf.image.random_crop(image, (200,200,3))
        image = tf.image.resize(image, size=(224, 224))
        
    # NOISE    
    if augment_op[4] == '1':
        aux_image = image + (noise * rand[4])
        image = tf.clip_by_value(aux_image, 0.0, 255)
    
    
    return image, features["category_id"]

# cria arquivos de valid
def prepare_sample_valid(features):
    image = tf.cast(features["image"], dtype=tf.float32)  
    image = tf.image.resize(image, size=(224, 224))   
    return image, features["category_id"]


def get_train_dataset(filenames, batch_size):
    dataset = (
        tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
        .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
        .map(prepare_sample_train, num_parallel_calls=AUTOTUNE)
        .shuffle(batch_size * 10)
        .batch(batch_size)
        .prefetch(AUTOTUNE)
    )
    return dataset

def get_valid_dataset(filenames, batch_size):
    dataset = (
        tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
        .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
        .map(prepare_sample_valid, num_parallel_calls=AUTOTUNE)
        .shuffle(batch_size * 10)
        .batch(batch_size)
        .prefetch(AUTOTUNE)
    )
    return dataset

In [ ]:
"""

train_ds = keras.utils.image_dataset_from_directory(
    directory='../dataset_varejo/oclusao/',
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=(50, 50), seed=10, validation_split=0.03, subset="training")

valid_ds = keras.utils.image_dataset_from_directory(
    directory='../dataset_varejo/oclusao/',
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=(50, 50), seed=10, validation_split=0.03, subset="validation")
    
"""


train_filenames = tf.io.gfile.glob(f"tfrecords_position/train/*.tfrec")

train_ds = get_train_dataset(train_filenames, batch_size)

for data in train_ds.take(3).as_numpy_iterator():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4))

    ax1.imshow(data[0][0] /255.)
    ax2.imshow(data[0][1] /255.)
    ax3.imshow(data[0][2] /255.)
   
    plt.show()


val_filenames = tf.io.gfile.glob(f"tfrecords_position/val/*.tfrec")

valid_ds = get_valid_dataset(val_filenames, batch_size)

for data in valid_ds.take(3).as_numpy_iterator():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4))

    ax1.imshow(data[0][0] /255.)
    ax2.imshow(data[0][1] /255.)
    ax3.imshow(data[0][2] /255.)
   
    plt.show()    
    
"""
def is_test(x, y):
    return x % 15 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

valid_ds = full_ds.enumerate().filter(is_test).map(recover)
train_ds = full_ds.enumerate().filter(is_train).map(recover)    
"""
print("ok")

In [ ]:
# get pre-trained model as backend
base_model = keras.applications.MobileNetV3Small(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

In [ ]:
from keras import Model
model = Model(inputs=base_model.input, outputs=base_model.get_layer('multiply_6').output)

# add a global spatial average pooling layer
x = model.output
x = keras.layers.Conv2D(64, (3,3), padding='valid', kernel_regularizer=keras.regularizers.l2(0.0005))(x)
x = keras.layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0005))(x)
# and a logistic layer 
predictions = keras.layers.Dense(5, activation='softmax', name="output_layer")(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

In [ ]:
model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                metrics=['sparse_categorical_accuracy'])


#loss='CategoricalCrossentropy', metrics=["accuracy"])
history = model.fit(train_ds, epochs=50, validation_data=valid_ds)

In [ ]:
#  "Accuracy"
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

# shuffle
import matplotlib.pyplot as plt

samples = valid_ds.take(3)


for batch in samples.as_numpy_iterator():
    images, labels = batch        
    
    for img, lbl in zip(images[:], labels[:]):
        #print(img.shape, lbl)
        
        o = model(np.expand_dims(img, 0))
        if tf.argmax(o, -1).numpy()[0] != lbl:            
            print(tf.argmax(o, -1).numpy()[0], lbl)
            plt.figure(figsize=(7, 7))
            plt.imshow(img/255.)
            plt.show()
    


In [ ]:
model.save("modelos/modelotf/")
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_layer' --saved_model_tags=serve "modelos/modelotf"  "modelos/web_model"

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

layer_name = "multiply"
extractor = keras.Model(inputs=model.inputs,outputs=model.get_layer(layer_name).output)
features = extractor(data[0][0:1])[0,...,0]

In [ ]:
print(features.numpy().shape)
print(features/np.max(features))

plt.imshow(features/np.max(features))
plt.show()

In [ ]:
model(train_ds.take(1).as_numpy_iterator())